In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from tqdm import tqdm
from cleanco import cleanco
import re
import os.path
import json

In [2]:
all_contacts_file = 'all-contacts_20191212.csv'
checked_domains_file = 'CheckedCompanies_20191212.csv'

In [3]:
after_underscore = checked_domains_file.find('_')+1
before_ext = checked_domains_file.find('.csv')
checked_file_date = checked_domains_file[after_underscore:before_ext]

In [4]:
all_contacts = pd.read_csv(all_contacts_file)
cols = ['Contact ID','Email','Company Name','Industry','Carrier Industry Segment','Intermediary Industry Segment','Shipper Industry Segment','Create Date']
all_contacts = all_contacts[cols]
all_contacts['Create Date'] = pd.to_datetime(all_contacts['Create Date'],errors='coerce').dt.date
all_contacts.sort_values('Create Date',ascending=False,inplace=True)
all_contacts.reset_index(drop=True,inplace=True)
contacts_array = all_contacts.to_numpy()
for i in tqdm(range(contacts_array.shape[0])):
    str_date = contacts_array[i][7].strftime('%Y%m%d')
    contacts_array[i][7] = str_date
    email = str(contacts_array[i][1]).lower().strip()
    where_at = email.find('@')
    domain = email[where_at:]
    contacts_array[i][1] = domain
    
    co_name = str(contacts_array[i][2]).lower().strip()
    co_name = re.sub(r"[(,.;@#?!&$)]+\ *"," ",co_name)
    co_name = ' '.join(co_name.split())
    contacts_array[i][2] = co_name

100%|██████████| 258499/258499 [00:05<00:00, 50065.38it/s]


In [5]:
before = all_contacts['Industry'].isnull().sum()
before

150474

In [6]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union
def string_to_list(string):
    string = re.sub(r"[(,.;@#?!&$)]+\ *", " ", string).lower()
    list = string.split()
    return list

In [7]:
d_segment_file,cn_segment_file = 'DictDomainSegments.json','DictCoNameSegments.json'
with open(d_segment_file,'r') as df, open(cn_segment_file,'r') as cnf:
    ds = json.load(df)
    cns = json.load(cnf)

In [8]:
contacts_array[300]

array([73440001, '@redwoodlogistics.com', 'redwood logistics',
       'Intermediary', nan, '3PL / Brokerage', nan, '20191210'],
      dtype=object)

In [9]:
contacts_array[10]

array([73472751, '@go3star.com', '3star logistics inc', 'Carrier',
       'Trucking', nan, nan, '20191212'], dtype=object)

In [10]:
contacts_array

array([[73473101, '@odysseylogistics.com', 'nan', ..., nan, nan,
        '20191212'],
       [73473051, '@gmail.com', 'pranatrucking', ..., nan, nan,
        '20191212'],
       [73471951, '@yahoo.com', 'nan', ..., nan, nan, '20191212'],
       ...,
       [11621, '@jefferies.com', 'jefferies', ..., nan, nan, '20181108'],
       [27742, '@gmail.com', 'kool pak llc', ..., nan, nan, '20181108'],
       [19410, '@schneider.com', 'schneider logistics', ...,
        '3PL / Brokerage', nan, '20181108']], dtype=object)

In [170]:
ds.get('@jbhunt.com')

['j b hunt transport services inc',
 'Intermediary',
 '3PL / Brokerage',
 '20191125',
 '20191212']

In [171]:
cns.get('j b hunt transport services inc')

['Intermediary', '3PL / Brokerage', '20191212']

In [172]:
cns.get('jb hunt')

['Intermediary', '3PL / Brokerage', '20191212']

In [173]:
# contacts_array = 
for i in tqdm(range(contacts_array.shape[0])):
    if pd.isnull(contacts_array[i][3]) and not pd.isnull(contacts_array[i][1]):
        domain = contacts_array[i][1]
        if domain in ds:
            info = ds.get(domain)  # info = [Company name, Industry, Sub-industry, checked_contact_date, checked_file_date]
            if info[1] == 'Carrier':
                contacts_array[i][2],contacts_array[i][3],contacts_array[i][4] = info[0],info[1],info[2]
            elif info[1] == 'Intermediary':
                contacts_array[i][2],contacts_array[i][3],contacts_array[i][5] = info[0],info[1],info[2]
            elif info[1] == 'Shipper/Retailer/Manufacturer/BCO':
                # more if-statements to change segmentation info to include commas 
                # this is because commas 
                if info[2] == 'Food Beverages & Tobacco':
                    contacts_array[i][2],contacts_array[i][3],contacts_array[i][6] = info[0],info[1],'Food, Beverages & Tobacco'
                elif info[2] == 'Hotels Restaurants & Leisure':
                    contacts_array[i][2],contacts_array[i][3],contacts_array[i][6] = info[0],info[1],'Hotels, Restaurants & Leisure'
                else:
                    contacts_array[i][2],contacts_array[i][3],contacts_array[i][6] = info[0],info[1],info[2]
            else:
                contacts_array[i][2],contacts_array[i][3] = info[0],info[1]
    if not pd.isnull(contacts_array[i][2]) and pd.isnull(contacts_array[i][3]) and not pd.isnull(contacts_array[i][1]):
        domain = contacts_array[i][1]
        if domain in ds:
            if ds.get(domain)[0] in contacts_array[i][2]:
                info = ds.get(domain)  # info = [Company name, Industry, Sub-industry, checked_contact_date, checked_file_date]
                if info[1] == 'Carrier':
                    contacts_array[i][3],contacts_array[i][4] = info[1],info[2]
                elif info[1] == 'Intermediary':
                    contacts_array[i][3],contacts_array[i][5] = info[1],info[2]
                elif info[1] == 'Shipper/Retailer/Manufacturer/BCO':
                    contacts_array[i][3],contacts_array[i][6] = info[1],info[2]
                else:
                    contacts_array[i][3] = info[0],info[1]

100%|██████████| 258499/258499 [00:01<00:00, 180349.96it/s]


In [174]:
segmented_contacts = pd.DataFrame(data=contacts_array)
segmented_contacts.columns = cols
after = segmented_contacts['Industry'].isnull().sum()
ds_check = before-after
print('Updated contacts after using DictDomainSegments: {}'.format(ds_check))

Updated contacts after using DictDomainSegments: 3153


In [175]:
for i in tqdm(range(contacts_array.shape[0])):
    if not pd.isnull(contacts_array[i][2]):# not pd.isnull(contacts_array[i][3]):
        coname = contacts_array[i][2]
        if coname in cns:
            info = cns.get(coname)  # info = [Industry, Sub-industry, checked_file_date]
            if info[0] == 'Carrier':
                contacts_array[i][3],contacts_array[i][4] = info[0],info[1]
            elif info[0] == 'Intermediary':
                contacts_array[i][3],contacts_array[i][5] = info[0],info[1]
            elif info[0] == 'Shipper/Retailer/Manufacturer/BCO':
                if info[1] == 'Food Beverages & Tobacco':
                    contacts_array[i][3],contacts_array[i][6] = info[0],'Food, Beverages & Tobacco'
                elif info[1] == 'Hotels Restaurants & Leisure':
                    contacts_array[i][3],contacts_array[i][6] = info[0],'Hotels, Restaurants & Leisure'
                else:
                    contacts_array[i][3],contacts_array[i][6] = info[0],info[1]
            else:
                contacts_array[3] = info[0]

100%|██████████| 258499/258499 [00:00<00:00, 395214.44it/s]


In [176]:
segmented_contacts = pd.DataFrame(data=contacts_array)
segmented_contacts.columns = cols
after = segmented_contacts['Industry'].isnull().sum()
cns_check = before-after
print('Updated contacts after using DictCoNameSegments: {}'.format(cns_check-ds_check))

Updated contacts after using DictCoNameSegments: 148


In [177]:
print('Total change: {}'.format(before-after))

Total change: 3301
